In [ ]:
"""
Import ISC vectors from hyperalignment_train_on_ctrl_tasks_test_on_storytelling.py,
generate statmaps, and make surface plots.
"""

In [ ]:
import numpy as np
import pandas as pd
from nilearn import image as nImage
from nilearn import input_data
from nilearn import datasets
from nilearn import surface
from nilearn import plotting

In [46]:
# results folder
resultsFolder = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/preprocessing/hyperalignment/results/'

# dataset parameters for loading
trainTask = 3 # 1=storytelling_independent, 2=storytelling_joint, 3=listening, 4=reading, 5=listening_&_reading
testCond = 1 # 0=independent, 1=joint
testTask = 1 # 0=all, 1=storytelling listening intervals, 2=storytelling reading intervals

# define hyperscanning task descriptions
taskDescrips = ['storytelling_independent',
                'storytelling_joint',
                'listening',
                'reading',
                'listening_&_reading']


baseFolder = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/preprocessing/hyperalignment/'
fileName = 'train_on_' + taskDescrips[trainTask-1] + '_test_on_' + taskDescrips[testCond]
saveFile = baseFolder + 'results/' + fileName
if testTask == 0:
    testTag = 'all'
elif testTask == 1:
    testTag = 'listening'
elif testTask == 2:
    testTag = 'speaking'
saveFile = saveFile + '_' + testTag

# set data files (0=pre-hyperalignment, 1=post-hyperalignment)
files = [saveFile + '_med_corr_pre_hyp.npy',
         saveFile + '_med_corr_post_hyp.npy']

labels = [[]] * 3
labels[0] = ['training data, pre-hyp', 'testing data, pre-hyp']
labels[1] = ['training data, post-hyp', 'testing data, post-hyp']
labels[2] = ['training data, diff', 'testing data, diff']

In [47]:
def surfaceStatMap(masker,statMapVec,avgSurface,thresh=0,conditions=['']):

    """
    :param masker:
    :param statMapVec:
    :param avgSurface:
    :param thresh:
    :param conditions: array of strings corresponding to different conditions
    :return:
    """

    # preallocate task arrays
    statMap = [[]] * len(conditions)
    texture = [[]] * len(conditions)
    view = [[]] * len(conditions)

    # for each task...
    for COND in range(len(conditions)):

        # get stat map
        statMap[COND] = masker.inverse_transform(statMapVec[COND])

        # surface plot
        texture[COND] = [[]] * 2
        view[COND] = [[]] * 2

        for HEMI in [0,1]:
            if HEMI == 0:
                texture[COND][HEMI] = surface.vol_to_surf(statMap[COND], avgSurface.pial_left)
                view[COND][HEMI] = plotting.view_surf(avgSurface.infl_left,
                                                                       texture[COND][HEMI],
                                                                       threshold=thresh,
                                                                       colorbar=True,
                                                                       title= conditions[COND] + ' left',
                                                                       bg_map=avgSurface.sulc_left)
            else:
                texture[COND][HEMI] = surface.vol_to_surf(statMap[COND], avgSurface.pial_right)
                view[COND][HEMI] = plotting.view_surf(avgSurface.infl_right,
                                                                       texture[COND][HEMI],
                                                                       threshold=thresh,
                                                                       colorbar=True,
                                                                       title=conditions[COND] + ', right',
                                                                       bg_map=avgSurface.sulc_right)

    return view


In [48]:
# get masker object and average surface mesh
maskFile = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/misc/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask_RESAMPLED.nii'
maskImg = nImage.load_img(maskFile)
masker = input_data.NiftiMasker(maskImg)
masker.fit_transform(maskImg)
fsaverage = datasets.fetch_surf_fsaverage()

In [49]:
# preallocate
medCorr = [[]] * 3
statMaps = [[]] * 3

# loop through results files
for HYP in range(len(files)):

    # load data
    medCorr[HYP] = list(np.load(files[HYP]))

    # make statmap
    statMaps[HYP] = surfaceStatMap(masker,medCorr[HYP],fsaverage,0,[labels[HYP][0], labels[HYP][1]])

# get pre-post difference
medCorr[2] = [[]] * 2
for DS in range(2):
    medCorr[2][DS] = medCorr[1][DS] - medCorr[0][DS]

# make difference statmap
statMaps[2] = surfaceStatMap(masker,medCorr[2],fsaverage,0,[labels[2][0], labels[2][1]])

/dartfs-hpc/rc/home/z/f00589z/.conda/envs/hypeScanCentral/lib/python3.9/site-packages/nilearn/plotting/html_document.py:56: UserWarning: It seems you have created more than 10 nilearn views. As each view uses dozens of megabytes of RAM, you might want to delete some of them.
  warnings.warn('It seems you have created more than {} '
/dartfs-hpc/rc/home/z/f00589z/.conda/envs/hypeScanCentral/lib/python3.9/site-packages/nilearn/plotting/html_document.py:56: UserWarning: It seems you have created more than 10 nilearn views. As each view uses dozens of megabytes of RAM, you might want to delete some of them.
  warnings.warn('It seems you have created more than {} '
/dartfs-hpc/rc/home/z/f00589z/.conda/envs/hypeScanCentral/lib/python3.9/site-packages/nilearn/plotting/html_document.py:56: UserWarning: It seems you have created more than 10 nilearn views. As each view uses dozens of megabytes of RAM, you might want to delete some of them.
  warnings.warn('It seems you have created more than {} 

(69880,)

In [50]:
# conditions
print(fileName)

# initialize dataframe to track the change in ISC pre-/post/hyperalignment in the training and testing datasets
meanMedCorrDiff = pd.DataFrame(columns=['Δ training set', 'Δ testing set'], index=range(1))

times = ['pre','post']
dataTypes = ['train','test']
i = 0
for DATA in [0,1]:
    for TIME in [0,1]:
        print(dataTypes[DATA] + ' set ' + times[TIME] + '-hyp: ' + str(np.round(np.mean(medCorr[TIME][DATA]),2)))
    meanMedCorrDiff.iloc[0,DATA] = np.round(np.mean(medCorr[1][DATA]) - np.mean(medCorr[0][DATA]),3)


train_on_listening_test_on_storytelling_joint
train set pre-hyp: 0.06
train set post-hyp: 0.28
test set pre-hyp: 0.07
test set post-hyp: 0.03


In [51]:
meanMedCorrDiff

,Δ training set,Δ testing set
0,0.217,-0.041


In [ ]:
statMaps[2][0][0]

In [ ]:
statMaps[2][0][1]

In [ ]:
statMaps[2][1][0]

In [ ]:
statMaps[2][1][1]
